## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [15]:
import json
import numpy as np
from openai import AsyncOpenAI
import pandas as pd
from pydantic import BaseModel
from tqdm.auto import tqdm

from lattereview.providers import OpenAIProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import ScoringReviewer
from lattereview.workflows import ReviewWorkflow

## Data

Building five example stories and dummy question-answering pairs from each story:

In [3]:
class BuildStoryOutput(BaseModel):
    story: str
    questions: list[str]
    answers: list[bool]

async def build_story():
    prompt = """
    Write a one-paragraph story with whatever realistic or imaginary theme you like,  
    then create three TRUE/FALSE questions based on your story. 
    Ensure that only readers of your story can determine whether the statements are true or false. 
    Do not reveal the answers to your questions.
    Return your story, a Python list of three questions, and another Python list of three boolean responses to the questions as your output.
    """
    provider = OpenAIProvider(model="gpt-4o", response_format_class=BuildStoryOutput)
    return await provider.get_json_response(prompt, temperature=0.9)

def run_build_story():
    response =  asyncio.run(build_story())[0]
    return response

data = {
    "question": [],
    "answer": [],
    "story": []
}
for i in tqdm(range(5)):
    out = json.loads(run_build_story())
    for j in range(3):
        data["question"].append(out["questions"][j])
        data["answer"].append(out["answers"][j])
        data["story"].append(out["story"])


data = pd.DataFrame(data)
data.to_csv("data.csv", index=False)
data

100%|██████████| 5/5 [00:15<00:00,  3.17s/it]


,question,answer,story
0,The old clockmaker's name is Elara.,True,In a quiet village nestled at the edge of an a...
1,The magical clocks were infused with moonlight...,False,In a quiet village nestled at the edge of an a...
2,Leo discovered the workshop on a quiet evening...,False,In a quiet village nestled at the edge of an a...
3,Is the magical carpet discovered by Omar said ...,False,In the heart of the bustling markets of Marrak...
4,Does Omar test the carpet's powers during the ...,False,In the heart of the bustling markets of Marrak...
5,Does the carpet lift into the air after Omar w...,True,In the heart of the bustling markets of Marrak...
6,Orin is a young rabbit who speaks all the anim...,False,"Once upon a time, in the enchanted forest of E..."
7,Pippin asked Orin for help finding his family ...,True,"Once upon a time, in the enchanted forest of E..."
8,The animals of Eldoria celebrated with a feast...,False,"Once upon a time, in the enchanted forest of E..."
9,Elara is a wise old owl living in the Enchante...,True,"In the heart of the Enchanted Forest, beneath ..."


Embedding the stories to build a vector base:

In [4]:
data = pd.read_csv("data.csv")

async def get_embedding(text):
    client = AsyncOpenAI()
    if isinstance(text, str):
        text = [text]
    text = [x.replace("\n", " ") for x in text]         
    out = await client.embeddings.create(
        model="text-embedding-ada-002",
        input=text,
        encoding_format="float"
    )
    out = [np.array(x.embedding) for x in out.data]
    return out if len(out) > 1 else out[0]

stories = {story: None for story in set(data["story"].tolist())}

# Create async tasks for all embeddings
async def process_embeddings():
    tasks = [get_embedding(story) for story in stories.keys()]
    embeddings = await asyncio.gather(*tasks)
    return list(zip(embeddings, stories.keys()))

# Run the async code and get results
vector_story_pairs = await process_embeddings()
vector_base = np.array([x[0] for x in vector_story_pairs])
vector_base

array([[ 0.01315026, -0.02092773, -0.02281476, ..., -0.00785609,
        -0.00963829, -0.02540943],
       [-0.00458142, -0.0099721 , -0.00658824, ...,  0.01933072,
        -0.00189261, -0.02981187],
       [ 0.01548435,  0.0108151 , -0.02071231, ...,  0.01972791,
         0.00259902, -0.03251181],
       [ 0.01010198, -0.00488657, -0.01606056, ...,  0.01285108,
        -0.00878004, -0.03740888],
       [ 0.00844919,  0.00391604, -0.00262109, ..., -0.00188088,
         0.02262308, -0.01502078]])

## Retrieval

In [5]:
async def find_relevant_story(statement):
    s_embeddings = await get_embedding(statement)
    dot_product = np.dot(vector_base, s_embeddings)
    base_norms = np.linalg.norm(vector_base, axis=1)
    query_norm = np.linalg.norm(s_embeddings)
    cosine_similarities = dot_product / (base_norms * query_norm)
    retrieved_index = np.argmax(cosine_similarities)
    retrieved_story = vector_story_pairs[retrieved_index][1]
    return retrieved_story

input_index = 11
statement = data.iloc[input_index]["question"]
retrieved_story = await find_relevant_story(statement)

print(f"=== The question was chosen from row {input_index} ===\n{statement}")
print(f"=== The related story to the question ===\n{data.iloc[input_index]['story']}")
print(f"=== The retrieved Story ===\n{retrieved_story}")

=== The question was chosen from row 11 ===
The oak trees were discussing the arrival of a traveler.
=== The related story to the question ===
In the heart of the Enchanted Forest, beneath a sky perpetually painted with twilight hues, lived a wise old owl named Elara. Elara had the unique ability to understand every language, including those of the whispers of the trees and the chatter of the brook. One day, as she perched atop the highest branch, she overheard a conversation in the wind between two ancient oak trees discussing the arrival of a mysterious traveler. Intrigued, Elara decided to keep a close watch from her aerial vantage point. By sunset, a cloaked figure appeared, ambling along the forest path, their footsteps soft as the rustling leaves. The traveler paused at the foot of Elara's tree, glanced upward, and said, "I seek the knowledge you have guarded for centuries." Elara blinked her large, golden eyes, recognizing this moment as one foretold by the stars long ago, and p

## Scoring with Retrieval Augmented Generation

In [13]:
reviewer = ScoringReviewer(
    provider=LiteLLMProvider(model="gpt-4o-mini"),
    name="reviewer",
    max_concurrent_requests=20, 
    backstory="A frequent book reader",
    input_description="TRUE/FALSE questions about stories",
    model_args={"max_tokens": 200, "temperature": 0.1},
    reasoning = "brief",
    scoring_task="Decide if the input statement is True or False given the provided story in the provided context",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the statement is TRUE and 2 if the statement is FALSE.',
    additional_context = find_relevant_story
)

review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [reviewer],
            "text_inputs": ["question"]
        }
    ]
)

updated_data = asyncio.run(review(data))
updated_data


====== Starting review round A (1/1) ======

Processing 15 eligible rows


['round: A', 'reviewer_name: reviewer'] -                     2024-12-28 14:30:38: 100%|██████████| 15/15 [00:05<00:00,  2.83it/s]

The following columns are present in the dataframe at the end of reviewer's reivew in round A: ['question', 'answer', 'story', 'round-A_reviewer_output', 'round-A_reviewer_reasoning', 'round-A_reviewer_score', 'round-A_reviewer_certainty']


,question,answer,story,round-A_reviewer_output,round-A_reviewer_reasoning,round-A_reviewer_score,round-A_reviewer_certainty
0,The old clockmaker's name is Elara.,True,In a quiet village nestled at the edge of an a...,{'reasoning': 'The statement claims that the o...,The statement claims that the old clockmaker's...,1,100
1,The magical clocks were infused with moonlight...,False,In a quiet village nestled at the edge of an a...,{'reasoning': 'The statement claims that the m...,The statement claims that the magical clocks w...,2,90
2,Leo discovered the workshop on a quiet evening...,False,In a quiet village nestled at the edge of an a...,{'reasoning': 'The statement claims that Leo d...,The statement claims that Leo discovered the w...,1,70
3,Is the magical carpet discovered by Omar said ...,False,In the heart of the bustling markets of Marrak...,{'reasoning': 'The statement claims that the m...,The statement claims that the magical carpet d...,2,90
4,Does Omar test the carpet's powers during the ...,False,In the heart of the bustling markets of Marrak...,{'reasoning': 'The statement asks if Omar test...,The statement asks if Omar tests the carpet's ...,2,100
5,Does the carpet lift into the air after Omar w...,True,In the heart of the bustling markets of Marrak...,{'reasoning': 'The statement is TRUE because t...,The statement is TRUE because the context desc...,1,95
6,Orin is a young rabbit who speaks all the anim...,False,"Once upon a time, in the enchanted forest of E...",{'reasoning': 'The statement claims that Orin ...,The statement claims that Orin is a young rabb...,2,95
7,Pippin asked Orin for help finding his family ...,True,"Once upon a time, in the enchanted forest of E...",{'reasoning': 'The statement is true because i...,The statement is true because it accurately re...,1,95
8,The animals of Eldoria celebrated with a feast...,False,"Once upon a time, in the enchanted forest of E...",{'reasoning': 'The statement claims that the a...,The statement claims that the animals of Eldor...,2,90
9,Elara is a wise old owl living in the Enchante...,True,"In the heart of the Enchanted Forest, beneath ...","{'reasoning': 'The statement ""Elara is a wise ...","The statement ""Elara is a wise old owl living ...",1,100


In [14]:
reviewer.memory[11]

{'system_prompt': "Your name is: <<reviewer>> Your backstory is: <<A frequent book reader>>. Your task is to review input itmes with the following description: <<TRUE/FALSE questions about stories>>. Your final output should have the following keys: reasoning (<class 'str'>), score (<class 'int'>), certainty (<class 'int'>).",
 'model_args': {'max_tokens': 200, 'temperature': 0.1},
 'input_prompt': '**Review the input item below and complete the scoring task as instructed:** --- **Input item:** <<Review Task ID: A-11 === question === The oak trees were discussing the arrival of a traveler.>> **Scoring task:** <<Decide if the input statement is True or False given the provided story in the provided context>> --- **Instructions:** 1. **Score** the input item using only the values in this set: [1, 2]. 2. Follow these rules when determining your score: <<Score 1 if the statement is TRUE and 2 if the statement is FALSE.>>. 3. After assigning a score, report your certainty level as a value b